In [ ]:
import pandas as pd
from PIL import Image, ImageDraw
import os

In [ ]:
# JSONL_FILE = "gpt-4o_UEye.jsonl"
# ERROR_FILE = "incorrect_4o_ueye.csv"
JSONL_FILE = "org_UEye.jsonl"
ERROR_FILE = "incorrect_org_ueye.csv"

df = pd.read_json(JSONL_FILE, lines=True)

df

In [ ]:
def in_bound(output, bbox):
    return output[0] >= bbox[0] and output[1] >= bbox[1] and output[0] <= bbox[0] + bbox[2] and output[1] <= bbox[1] + bbox[3]

df["correct"] = df.apply(lambda x: in_bound(x["output"], x["bbox"]), axis=1)
df["bbox_xyxy"] = df["bbox"].apply(lambda x: [x[0], x[1], x[0] + x[2], x[1] + x[3]])

all_count = df.shape[0]
correct_count = df[df["correct"] == True].shape[0]
incorrect_count = df[df["correct"] == False].shape[0]
accuracy = correct_count / all_count
print(accuracy)

df_incorrect = df[df["correct"] == False]
df_incorrect.to_csv(ERROR_FILE)
df_incorrect

In [ ]:
for i in range(250, df_incorrect.shape[0]):
    row = df_incorrect.iloc[i]
    img = Image.open(os.path.join("..", "screenspot_imgs", row["img_filename"]))
    img_draw = ImageDraw.Draw(img)
    img_draw.rectangle(row["bbox_xyxy"], outline="red", width=3)
    img_draw.circle(row["output"], radius=6, fill="blue")
    
    print("img_filename:", row["img_filename"])
    try:
        print("Instruction:", row["instruction"])
    except Exception as e:
        pass
    print("Data Type:", row["data_type"])
    print("Data source:", row["data_source"])
    print("Platform:", row["platform"])
    print("Scale:", row["scale"])
    print("Description:", row["description"])
    display(img)